Nguyễn Hữu Quyến - 19522113

Lab 5



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px

df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/Data_Mining/Thuc_hanh/job-market.csv")

accounting_jobs = df[df['Classification'] == 'Accounting']

subsector_counts = accounting_jobs['SubClassification'].value_counts()

fig = px.bar(x=subsector_counts.index, y=subsector_counts.values, labels={'x':'Sub-sector', 'y':'Count'}, color=subsector_counts.index)
fig.update_layout(title='Accounting')
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data for the month of October 2018
df = df[(df['Date'] >= '2018-10-01') & (df['Date'] <= '2018-10-31')]

# Group the data by day and count the job postings
job_postings_by_day = df.groupby(pd.Grouper(key='Date', freq='D'))['Title'].count()

# Create a line graph of the job postings by day using Matplotlib
plt.plot(job_postings_by_day.index, job_postings_by_day.values)
plt.xticks(job_postings_by_day.index, [d.strftime('%d.%b') for d in job_postings_by_day.index], rotation=45)
plt.xlabel('Label')
plt.ylabel('Count')
plt.title('Job Postings by Date')
plt.show()

Section 2: Moving Beyond Static Visualizations

In [ ]:
import pandas as pd
questions_per_library = pd.read_csv(
'/content/drive/MyDrive/Colab_Notebooks/Data_Mining/Thuc_hanh/data/data/stackoverflow.zip', parse_dates=True, index_col='creation_date'
).loc[:, 'pandas': 'bokeh'].resample ('1M').sum().cumsum().reindex( pd.date_range('2008-08', '2021-10', freq= 'M')
).fillna(0)
questions_per_library.tail()

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import ticker
def bar_plot(data):
  fig, ax = plt.subplots (figsize=(8, 6))
  sort_order = data.last('1M').squeeze().sort_values().index 
  bars = [
    bar.set_label(label) for label, bar in
    zip(sort_order, ax.barh (sort_order, [0] * data.shape[1]))
]

  ax.set_xlabel('total questions', fontweight='bold')
  ax.set_xlim(0, 250_000)
  ax.xaxis.set_major_formatter (ticker. EngFormatter()) 
  ax.xaxis.set_tick_params (labelsize=12) 
  ax.yaxis.set_tick_params (labelsize=12)
  for spine in ['top', 'right']:
    ax.spines[spine].set_visible (False)
  fig.tight_layout()
  return fig, ax

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
bar_plot(questions_per_library)

In [ ]:
def generate_plot_text(ax):
  annotations = [
    ax.annotate(
      '', xy=(0, bar.get_y() + bar.get_height()/2), 
      ha='left', va='center'
    ) for bar in ax.patches
]

  time_text = ax.text(
      0.9, 0.1,'', transform=ax.transAxes,
      fontsize=15, ha='center', va='center'
  )

  return annotations, time_text

In [ ]:
def update (frame,*, ax, df, annotations, time_text): 
  data=df.loc[frame, :]
# update bars
  for rect, text in zip(ax.patches, annotations):
    col = rect.get_label()
    if data[col]:
      rect.set_width (data[col])
      text.set_x(data[col])
      text.set_text(f' {data[col]:,.0f}')
  #update time
  time_text.set_text(frame.strftime('\b\n\Y'))

In [ ]:
from functools import partial
def bar_plot_init(questions_per_library): 
  fig, ax = bar_plot (questions_per_library)
  annotations, time_text = generate_plot_text(ax)
  
  bar_plot_update = partial(

    update, ax=ax, df=questions_per_library, 
    annotations=annotations, time_text=time_text
  )

  return fig, bar_plot_update

In [ ]:
fig, update_func = bar_plot_init(questions_per_library)
ani = FuncAnimation(
  fig, update_func, frames=questions_per_library.index, repeat=False
)
ani.save(
    '../media/stackoverflow_questions.mp4', 
    writer='ffmpeg', fps=10, bitrate=100, dpi=300
)

plt.close()


In [ ]:
from IPython import display
display.Video(
'../media/stackoverflow_questions.mp4', width=600, height=400, embed=True, html_attributes='controls muted autoplay'
)

Animating distributions over time

In [ ]:
subway = pd.read_csv(
    '/content/drive/MyDrive/Colab_Notebooks/Data_Mining/Thuc_hanh/data/data/NYC_subway_daily.csv', parse_dates=['Datetime'],
    index_col=['Borough', 'Datetime']
)
subway_daily= subway.unstack(0)
subway_daily.head()

In [ ]:
manhattan_entries = subway_daily['Entries']['M']

In [ ]:
import numpy as np
count_per_bin, bin_ranges = np.histogram(manhattan_entries,bins=30)

In [ ]:
def subway_histogram(data,bins, date_range):
    _,bin_ranges = np.histogram(data, bins=bins)
    weekday_mask= data.index.weekday <5
    configs = [
    {'label':'Weekend','mask': ~weekday_mask, 'ymax': 60},
    {'label': 'Weekday', 'mask':weekday_mask, 'ymax': 120}
    ]

    fig, axes = plt.subplots(1, 2, figsize=(8, 4), sharex=True)
    for ax, config in zip (axes, configs):

      _, _,config['hist'] = ax.hist(
      data[config['mask']].loc[date_range], bin_ranges, ec='black'
      )
      ax.xaxis.set_major_formatter (ticker. EngFormatter())
      ax.set(
        xlim=(0, None), ylim=(0, config['ymax']), 
        xlabel=f' {config["label"]} Entries'
      )

      for spine in ['top', 'right']:
        ax.spines [spine].set_visible (False)
        axes[0].set_ylabel('Frequency')
        fig.suptitle('Histogram of Daily Subway Entries in Manhattan')
        fig.tight_layout()
    return fig, axes, bin_ranges, configs

In [ ]:
_ = subway_histogram(manhattan_entries, bins=30, date_range='2017')

In [ ]:
def add_time_text(ax):
  time_text = ax.text(
      0.15, 0.9, '', transform=ax.transAxes,
      fontsize=15, ha='center', va='center'
  )
  return time_text

In [ ]:
def update(frame, *, data, configs, time_text, bin_ranges):
   artists = []

   time=frame.strftime('%b\n%Y')
   if time != time_text.get_text():
     time_text.set_text(time)
     artists.append(time_text)

   for config in configs:
     time_frame_mask = \
        (data.index > frame -pd.Timedelta(days=365)) & (data.index <= frame)
     counts, _= np.histogram(
         data[time_frame_mask & config['mask']],
         bin_ranges
     )
     for count, rect in zip(counts, config['hist'].patches):
       if count != rect.get_height():
            rect.set_height(count)
            artists.append(rect)

   return artists

In [ ]:
def histogram_init(data, bins, initial_date_range):
  fig, axes, bin_ranges, configs = subway_histogram(data, bins, initial_date_range)

  update_func= partial(
      update, data=data, configs=configs,
      time_text=add_time_text(axes[0]),
      bin_ranges=bin_ranges
  )

  return fig, update_func

In [ ]:
fig, update_func = histogram_init(
  manhattan_entries, bins=30, initial_date_range=slice('2017', '2019-07')

)
ani = FuncAnimation(
  fig, update_func, frames=manhattan_entries['2019-08':'2021'].index,
  repeat=False, blit=True

)
ani.save(
    '../media/subway_entries_subplots.mp4', 
    writer='ffmpeg', fps=30, bitrate=500, dpi=300
)

plt.close()

In [ ]:
from IPython import display
display.Video(
'../media/subway_entries_subplots.mp4', width=600, height=400, embed=True, html_attributes='controls muted autoplay'
)

Animating geospatial data with HoloViz

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
import pandas as pd

earthquakes = gpd.read_file('/content/drive/MyDrive/Colab_Notebooks/Data_Mining/Thuc_hanh/data/data/earthquakes.geojson').assign(
    time = lambda x: pd.to_datetime(x.time, unit='ms'),
    month = lambda x: x.time.dt.month
)[['geometry', 'mag', 'time', 'month']]

earthquakes.shape

In [ ]:
earthquakes.head()

In [ ]:
!pip install geoviews

In [ ]:
import geoviews as gv
import geoviews.feature as gf
import holoviews as hv

gv.extension('matplotlib')

In [ ]:
import calendar

def plot_earthquakes(data, month_num):
  points = gv.Points(
      data.query(f'month == {month_num}'),
      kdims = ['longitude', 'latitude'],
      vdims = ['mag']
  ).redim.range(mag = (-2, 10), latitude = (-90, 90))

  overlay = gf.land * gf.coastline * gf.borders * points

  return overlay.opts(
      gv.opts.Points(color = 'mag', cmap = 'fire_r', colorbar = True, alpha = 0.75),
      gv.opts.Overlay(
          global_extent = False, title = f'{calendar.month_name[month_num]}', fontscale = 2
      )
  )

In [ ]:
plot_earthquakes(earthquakes, 1).opts(
    fig_inches=(6, 3), aspect=2, fig_size=250, fig_bounds=(0.07, 0.05, 0.87, 0.95)
)

In [ ]:
frames = {
    month_num: plot_earthquakes(earthquakes, month_num)
    for month_num in range(1, 13)
}
holomap = hv.HoloMap(frames)

In [ ]:
hv.output(
    holomap.opts(
        fig_inches=(6, 3), aspect=2, fig_size=250,
        fig_bounds=(0.07, 0.05, 0.87, 0.95)
    ), holomap='gif', fps=5
)

In [ ]:
hv.save(
    holomap.opts(
        fig_inches=(6, 3), aspect=2, fig_size=250,
        fig_bounds=(0.07, 0.05, 0.87, 0.95)
    ), 'earthquakes.gif', fps=5
)

Section 3: Building Interactive Visualizations for Data Exploration

1. Read in and prepare the data.

In [ ]:
import geopandas as gpd
import pandas as pd

earthquakes = gpd.read_file('/content/drive/MyDrive/Colab_Notebooks/Data_Mining/Thuc_hanh/data/data/earthquakes.geojson').assign(
    time=lambda x: pd.to_datetime(x.time, unit='ms'),
    month=lambda x: x.time.dt.month
).dropna()

earthquakes.head()

2. Import the required libraries and set up the Bokeh backend.

In [ ]:
from cartopy import crs
import geoviews as gv
import geoviews.feature as gf

gv.extension('bokeh')

3. Create an overlay with tooltips and a slider.

In [ ]:
points = gv.Points(
    earthquakes,
    kdims=['longitude', 'latitude'],
    vdims=['month', 'place', 'tsunami', 'mag', 'magType']
)

# set colorbar limits for magnitude and axis limits
points = points.redim.range(
    mag=(-2, 10), longitude=(-180, 180), latitude=(-90, 90)
)

In [ ]:
overlay = gf.land * gf.coastline * gf.borders * points.groupby('month')

In [ ]:
interactive_map = overlay.opts(
    gv.opts.Feature(projection=crs.PlateCarree()),
    gv.opts.Overlay(width=700, height=450),
    gv.opts.Points(color='mag', cmap='fire_r', colorbar=True, tools=['hover'])
)

4. Render the visualization.

In [ ]:
import panel as pn

earthquake_viz = pn.panel(interactive_map, widget_location='bottom')

In [ ]:
earthquake_viz.embed()

Linking plots


1. Isolate the January earthquake data and prepare it for plotting.

In [ ]:
january_earthquakes = earthquakes.query('month == 1').assign(
    longitude=lambda x: x.geometry.x,
    latitude=lambda x: x.geometry.y
).drop(columns=['month', 'geometry'])

2. Enable the use of hvPlot for interactive plotting with pandas.

In [ ]:
!pip install panel hvplot

In [ ]:
import hvplot.pandas


3. Build a layout composed of an interactive map and a table with hvPlot.

In [ ]:
geo = january_earthquakes.hvplot(
    x='longitude', y='latitude', kind='points',
    color='mag', cmap='fire_r', clim=(-2, 10),
    tiles='CartoLight', geo=True, global_extent=True,
    xlabel='Longitude', ylabel='Latitude', title='January 2020 Earthquakes',
    frame_height=450
)

In [ ]:
table = january_earthquakes.sort_values(['longitude', 'latitude']).hvplot(
    kind='table', width=650, height=450, title='Raw Data'
)

4. Link selections across the visualizations in the layout.

In [ ]:
import holoviews as hv

selection = hv.link_selections.instance()
map_and_table_tabs = selection(layout).opts(tabs=True)

Additional plot types

In [ ]:
import numpy as np
import pandas as pd


flight_stats = pd.read_csv(
    '/content/drive/MyDrive/Colab_Notebooks/Data_Mining/Thuc_hanh/data/data/T100_MARKET_ALL_CARRIER.zip',
    usecols=[
        'CLASS', 'REGION', 'UNIQUE_CARRIER_NAME', 'ORIGIN_CITY_NAME', 'ORIGIN', 
        'DEST_CITY_NAME', 'DEST', 'PASSENGERS', 'FREIGHT', 'MAIL'
    ]
).rename(lambda x: x.lower(), axis=1).assign(
    region=lambda x: x.region.replace({
        'D': 'Domestic', 'I': 'International', 'A': 'Atlantic', 
        'L': 'Latin America', 'P': 'Pacific', 'S': 'System'
    }),
    route=lambda x: np.where(
        x.origin < x.dest,
        x.origin + '-' + x.dest,
        x.dest + '-' + x.origin
    )
)

In [ ]:
flight_stats.head()

In [ ]:
cities = [
    'Atlanta, GA', 'Chicago, IL', 'New York, NY', 'Los Angeles, CA',
    'Dallas/Fort Worth, TX', 'Denver, CO', 'Houston, TX', 
    'San Francisco, CA', 'Seattle, WA', 'Orlando, FL'
]

top_airlines = [
    'American Airlines Inc.', 'Delta Air Lines Inc.', 'JetBlue Airways',
    'Southwest Airlines Co.', 'United Air Lines Inc.'
]

Chord diagram

In [ ]:
total_flight_stats = flight_stats.query(
    '`class` == "F" and origin_city_name != dest_city_name'
    f' and origin_city_name.isin({cities}) and dest_city_name.isin({cities})'
).groupby([
    'origin', 'origin_city_name', 'dest', 'dest_city_name'
])[['passengers', 'freight', 'mail']].sum().reset_index().query('passengers > 0')

In [ ]:
total_flight_stats.sample(10, random_state=1)

In [ ]:
chord = hv.Chord(
    total_flight_stats,
    kdims=['origin', 'dest'], 
    vdims=['passengers', 'origin_city_name', 'dest_city_name', 'mail', 'freight'])

3. Customize the tooltips using Bokeh.

In [ ]:
from bokeh.models import HoverTool

tooltips = {
    'Source': '@origin_city_name (@origin)',
    'Target': '@dest_city_name (@dest)',
    'Passengers': '@passengers{0,.}',
    'Mail': '@mail{0,.} lbs.',
    'Freight': '@freight{0,.} lbs.',
}
hover = HoverTool(tooltips=tooltips)

Sankey plot

1. Isolate flight statistics for top routes.

In [ ]:
top_cities = cities[:5]

domestic_passenger_travel = flight_stats.query(
    'region == "Domestic" and `class` == "F" and origin_city_name != dest_city_name '
    f'and origin_city_name.isin({top_cities}) and dest_city_name.isin({top_cities})'
).groupby([
    'region', 'unique_carrier_name', 'route', 'origin_city_name', 'dest_city_name'
]).passengers.sum().reset_index()

domestic_passenger_travel.head()

2. Convert the data into a set of edges

In [ ]:
domestic_passenger_travel.unique_carrier_name = (
    domestic_passenger_travel.unique_carrier_name.replace(
        '^(?!' + '|'.join(top_airlines) + ').*$',
        'Other Airlines',
        regex=True
    )
)

In [ ]:
domestic_passenger_travel.groupby('unique_carrier_name').passengers.sum().div(
    domestic_passenger_travel.passengers.sum()
)

In [ ]:
def get_edges(data, *, source_col, target_col):
    aggregated = data.groupby([source_col, target_col]).passengers.sum()
    return aggregated.reset_index().rename(
        columns={source_col: 'source', target_col: 'target'}
    ).query('passengers > 0')

In [ ]:
carrier_edges = get_edges(
    domestic_passenger_travel, 
    source_col='region',
    target_col='unique_carrier_name'
).replace('^Domestic$', 'Top Routes', regex=True)

carrier_edges

In [ ]:
carrier_to_route_edges = get_edges(
    domestic_passenger_travel,
    source_col='unique_carrier_name',
    target_col='route'
)

carrier_to_route_edges.sample(10, random_state=1)

In [ ]:
all_edges = pd.concat([carrier_edges, carrier_to_route_edges]).assign(
    passengers=lambda x: x.passengers / 1e6
)

4. Create the Sankey plot with HoloViews.

In [ ]:
sankey = hv.Sankey(
    all_edges, 
    kdims=['source', 'target'],
    vdims=hv.Dimension('passengers', unit='M')
).opts(
    labels='index', label_position='right', cmap='Set1', # node config 
    edge_color='lightgray', # edge config
    width=750, height=600, # plot size config
    title='Travel Between the Top 5 Cities in 2019'
)

In [ ]:
sankey